In [1]:
import time 
from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
def shift(stack, queue, graph):
    """
    Shift the first word in the queue onto the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    stack = [queue[0]] + stack
    queue = queue[1:]
    return stack, queue, graph


def reduce(stack, queue, graph):
    """
    Remove the first item from the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    return stack[1:], queue, graph


def right_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the top of the stack to the first in the queue
    and shifts
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][queue[0]['id']] = stack[0]['id']
    if deprel:
        graph['deprels'][queue[0]['id']] = deprel
    else:
        graph['deprels'][queue[0]['id']] = queue[0]['deprel']
    return shift(stack, queue, graph)


def left_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the first in the queue to the top of the stack
    and reduces it.
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][stack[0]['id']] = queue[0]['id']
    if deprel:
        graph['deprels'][stack[0]['id']] = deprel
    else:
        graph['deprels'][stack[0]['id']] = stack[0]['deprel']
    return reduce(stack, queue, graph)


def can_reduce(stack, graph):
    """
    Checks that the top of the stack has a head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['id'] in graph['heads']:
        return True
    else:
        return False


def can_leftarc(stack, graph):
    """
    Checks that the top of the has no head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['id'] in graph['heads']:
        return False
    else:
        return True


def can_rightarc(stack):
    """
    Simply checks there is a stack
    :param stack:
    :return:
    """
    if not stack:
        return False
    else:
        return True


def empty_stack(stack, graph):
    """
    Pops the items in the stack. If they have no head, they are assigned
    a ROOT head
    :param stack:
    :param graph:
    :return:
    """
    for word in stack:
        if word['id'] not in graph['heads']:
            graph['heads'][word['id']] = '0'
            graph['deprels'][word['id']] = 'ROOT'
    stack = []
    return stack, graph


def equal_graphs(sentence, graph):
    """
    Checks that the graph corresponds to the gold standard annotation of a sentence
    :param sentence:
    :param graph:
    :return:
    """
    equal = True
    for word in sentence:
        if word['id'] in graph['heads'] and word['head'] == graph['heads'][word['id']]:
            pass
        else:
            #print(word, flush=True)
            equal = False
    return equal


In [3]:
def reference(stack, queue, graph):
    """
    Gold standard parsing
    Produces a sequence of transitions from a manually-annotated corpus:
    sh, re, ra.deprel, la.deprel
    :param stack: The stack
    :param queue: The input list
    :param graph: The set of relations already parsed
    :return: the transition and the grammatical function (deprel) in the
    form of transition.deprel
    """
    # Right arc
    if stack and stack[0]['id'] == queue[0]['head']:
        # print('ra', queue[0]['deprel'], stack[0]['cpostag'], queue[0]['cpostag'])
        deprel = '.' + queue[0]['deprel']
        stack, queue, graph = right_arc(stack, queue, graph)
        return stack, queue, graph, 'ra' + deprel
    # Left arc
    if stack and queue[0]['id'] == stack[0]['head']:
        # print('la', stack[0]['deprel'], stack[0]['cpostag'], queue[0]['cpostag'])
        deprel = '.' + stack[0]['deprel']
        stack, queue, graph = left_arc(stack, queue, graph)
        return stack, queue, graph, 'la' + deprel
    # Reduce
    if stack and can_reduce(stack, graph):
        for word in stack:
            if (word['id'] == queue[0]['head'] or
                        word['head'] == queue[0]['id']):
                # print('re', stack[0]['cpostag'], queue[0]['cpostag'])
                stack, queue, graph = reduce(stack, queue, graph)
                return stack, queue, graph, 're'
    # Shift
    # print('sh', [], queue[0]['cpostag'])
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'


def extract_features(formatted_corpus, mode, test_mode=False, vec=None, classifier=None):
    # EXTRACT FEATURES
    feature_names_1 = ['stack0_POS', 'stack0_word',
                       'queue0_POS', 'queue0_word',
                       'can-re', 'can-la']
    feature_names_2 = ['stack1_POS', 'stack1_word',
                       'queue1_POS', 'queue1_word']
    feature_names_3 = ['left_POS', 'left_word',
                       'right_POS', 'right_word']

    feature_names = {'mode1': feature_names_1, 'mode2': feature_names_2, 'mode3': feature_names_3}
    X = list()
    transitions = list()
    sent_cnt = 0
    for sentence in formatted_corpus:
        sent_cnt += 1
        # if sent_cnt % 1000 == 0:
        #    print(sent_cnt, 'sentences on', len(formatted_corpus), flush=True)
        stack = []
        queue = list(sentence)
        graph = {}
        graph['heads'] = {}
        graph['heads']['0'] = '0'
        graph['deprels'] = {}
        graph['deprels']['0'] = 'ROOT'

        while queue:
            if mode == 3:
                X_row = extract_mode_3(stack, queue, graph, feature_names, sentence)
            elif mode == 2:
                X_row = extract_mode_2(stack, queue, graph, feature_names, sentence)
            elif mode == 1:
                X_row = extract_mode_1(stack, queue, graph, feature_names, sentence)
            if not test_mode:
                stack, queue, graph, trans = reference(stack, queue, graph)
            elif test_mode:
                X_row_vec = vec.transform(X_row)
                trans_nr = classifier.predict(X_row_vec)
                stack, queue, graph, trans = parse_ml(stack, queue, graph, trans_nr[0])
            X.append(X_row)
            transitions.append(trans)
        stack, graph = empty_stack(stack, graph)
        # print('Equal graphs:', transition.equal_graphs(sentence, graph))

        # Poorman's projectivization to have well-formed graphs.
        if test_mode:
            for word in sentence:
                word['head'] = graph['heads'][word['id']]
                word['deprel'] = graph['deprels'][word['id']]
        # print(graph)
    # print(X)
    # print(transitions)
    if test_mode:
        save('out_{}_mode_{}.conll'.format("test", mode), formatted_corpus, column_names_2006)
    return X, transitions


def parse_ml(stack, queue, graph, trans):
    #right arc
    if stack and trans[:2] == 'ra' and can_rightarc(stack):
        stack, queue, graph = right_arc(stack, queue, graph, trans[3:])
        return stack, queue, graph, 'ra'
    #left arc
    if stack and trans[:2] == 'la' and can_leftarc(stack, graph):
        stack, queue, graph = left_arc(stack, queue, graph, trans[3:])
        return stack, queue, graph, 'la'
    #reduce
    if stack and trans[:2] == 're' and can_reduce(stack, graph):
        stack, queue, graph = reduce(stack, queue, graph)
        return stack, queue, graph, 're'
    #shift
    if stack and trans[:2] == 'sh':
        stack, queue, graph = shift(stack, queue, graph)
        return stack, queue, graph, 'sh'
    #action not possible -> shift
    else:
        stack, queue, graph = shift(stack, queue, graph)
        return stack, queue, graph, 'sh'


def train_model(X_dict, y, classifier, vec):
    start_time = time.clock()
    X = vec.fit_transform(X_dict)
    print("Training the model...")
    model = classifier.fit(X, y)
    # print(model)
    y_pred = classifier.predict(X)
    print(classification_report(y, y_pred))
    stop_time = time.clock()
    print("Training time:", (stop_time - start_time) / 60)


def test_model(X_test_dict, y_test_dict, classifier, vec):
    start_time = time.clock()
    X_test = vec.transform(X_test_dict)
    y_test_predicted = classifier.predict(X_test)
    #print("Classification report for classifier %s:\n%s\n"
    #      % (classifier, metrics.classification_report(y_test_dict, y_test_predicted)))
    stop_time = time.clock()
    print("Testing time:", (stop_time - start_time) / 60)

In [4]:
SIZE_1 = 6
SIZE_2 = 10
SIZE_3 = 14


def extract_mode_1(stack, queue, graph, feature_names, sentence=None):
    features = list()
    if stack:
        stack0 = stack[0]
        features.extend([stack0.get('postag'), stack0.get('form')])
    else:
        features.extend(['nil', 'nil'])
    if queue:
        queue0 = queue[0]
        features.extend([queue0.get('postag'), queue0.get('form')])
    else:
        features.extend(['nil', 'nil'])

    features.append(can_reduce(stack, graph))
    features.append(can_leftarc(stack, graph))
    return dict(zip(feature_names.get('mode1'), features))


def extract_mode_2(stack, queue, graph, feature_names, sentence=None):
    extracted_features = dict()
    dict_mode_1 = extract_mode_1(stack, queue, graph, feature_names)
    extracted_features.update(dict_mode_1)
    features_mode_2 = list()
    if len(stack) > 1:
        stack1 = stack[1]
        features_mode_2.extend([stack1.get('postag'), stack1.get('form')])
    else:
        features_mode_2.extend(['nil', 'nil'])
    if len(queue) > 1:
        queue1 = queue[1]
        features_mode_2.extend([queue1.get('postag'), queue1.get('form')])
    else:
        features_mode_2.extend(['nil', 'nil'])
    extracted_features.update(dict(zip(feature_names.get('mode2'), features_mode_2)))
    return extracted_features


def extract_mode_3(stack, queue, graph, feature_names, sentence=None):
    extracted_features = dict()
    dict_mode_2 = extract_mode_2(stack, queue, graph, feature_names)
    extracted_features.update(dict_mode_2)
    features_mode_3 = list(['nil']*4)
    if stack:
        id0 = int(stack[0].get('id'))
        for pos, word in enumerate(sentence):
            if word.get('id') == str(id0-1):
                features_mode_3[:2] = [word.get('postag'), word.get('form')]
            elif word.get('id') == str(id0+1):
                features_mode_3[2:] = [word.get('postag'), word.get('form')]
    extracted_features.update(dict(zip(feature_names.get('mode3'), features_mode_3)))
    return extracted_features

In [5]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    Recursive version
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        path = dir + '/' + file
        if os.path.isdir(path):
            files += get_files(path, suffix)
        elif os.path.isfile(path) and file.endswith(suffix):
            files.append(path)
    return files


def read_sentences(file):
    """
    Creates a list of sentences from the corpus
    Each sentence is a string
    :param file:
    :return:
    """
    f = open(file).read().strip()
    sentences = f.split('\n\n')
    return sentences


def split_rows(sentences, column_names):
    """
    Creates a list of sentence where each sentence is a list of lines
    Each line is a dictionary of columns
    :param sentences:
    :param column_names:
    :return:
    """
    new_sentences = []
    root_values = ['0', 'ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT', '0', 'ROOT', '0', 'ROOT']
    start = [dict(zip(column_names, root_values))]
    for sentence in sentences:
        rows = sentence.split('\n')
        sentence = [dict(zip(column_names, row.split())) for row in rows if row[0] != '#']
        sentence = start + sentence
        new_sentences.append(sentence)
    return new_sentences


def save(file, formatted_corpus, column_names):
    f_out = open(file, 'w')
    for sentence in formatted_corpus:
        for row in sentence[1:]:
            # print(row, flush=True)
            for col in column_names[:-1]:
                if col in row:
                    f_out.write(row[col] + '\t')
                else:
                    f_out.write('_\t')
            col = column_names[-1]
            if col in row:
                f_out.write(row[col] + '\n')
            else:
                f_out.write('_\n')
        f_out.write('\n')
    f_out.close()

In [6]:
train_file = 'swedish_talbanken05_train.conll'
test_file = 'swedish_talbanken05_test_blind.conll'
column_names_2006 = ['id', 'form', 'lemma', 'cpostag', 'postag', 'feats', 'head', 'deprel', 'phead', 'pdeprel']
column_names_2006_test = ['id', 'form', 'lemma', 'cpostag', 'postag', 'feats']

for i in range(1, 4):
    print("*"*100)
    print("MODE" + str(i))
    print("-"*100)
    vec = DictVectorizer(sparse=True)
    classifier = linear_model.LogisticRegression(penalty='l2', dual=True, solver='liblinear', verbose=1)
    # classifier = linear_model.Perceptron(penalty='l2')
    # classifier = tree.DecisionTreeClassifier()
    # classifier = linear_model.SGDClassifier(penalty='l2')
    sentences = read_sentences(train_file)
    sentences_test = read_sentences(test_file)
    formatted_corpus = split_rows(sentences, column_names_2006)
    formatted_corpus_test = split_rows(sentences_test, column_names_2006_test)

    X, y = extract_features(formatted_corpus, mode=i)

    train_model(X, y, classifier, vec)
    X_test, y_test = extract_features(formatted_corpus_test, mode=i, test_mode=True, vec=vec, classifier=classifier)
    test_model(X_test, y_test, classifier, vec)
    print("-"*100)
    print("DONE MODE" + str(i))
    print("*"*100)

****************************************************************************************************
MODE1
----------------------------------------------------------------------------------------------------
Training the model...


/home/miquel/.local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]

/home/miquel/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       la.++       0.92      0.94      0.93      7254
       la.+A       0.90      0.84      0.87       636
       la.+F       0.00      0.00      0.00        14
       la.AA       0.79      0.85      0.82      4749
       la.AG       1.00      0.40      0.57        25
       la.AN       0.00      0.00      0.00         6
       la.AT       0.98      0.99      0.98      7457
       la.BS       0.00      0.00      0.00         2
       la.C+       0.00      0.00      0.00         1
       la.CA       0.85      0.88      0.87       860
       la.DT       0.95      0.96      0.95     18300
       la.EO       0.00      0.00      0.00         1
       la.ES       0.00      0.00      0.00         5
       la.ET       0.00      0.00      0.00         2
       la.FO       1.00      0.60      0.75        50
       la.FS       0.93      0.37      0.53       594
       la.FV       0.00      0.00      0.00         3
       la.IC       0.82    